In [1]:
import json 
import pandas as pd
import datetime
from pandas.io.json import json_normalize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split 

[nltk_data] Downloading package stopwords to /Users/ankit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ankit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df=pd.read_json('uspto.json',lines=True)

In [3]:
objectdata=df.object.dropna().apply(pd.Series)

## Labelling the data as per their status

In [4]:
status=objectdata.status.unique()

In [5]:
approved=[status[0],status[2],status[5]]
approved

['Publications -- Issue Fee Payment Verified',
 'Awaiting TC Resp., Issue Fee Not Paid',
 'Patented Case']

In [6]:
disapproval=[status[1],status[3],status[7],status[8],status[10],status[11],status[16],status[18]]
disapproval

['Non Final Action Mailed',
 'Final Rejection Mailed',
 'Abandoned  --  Failure to Respond to an Office Action',
 'Advisory Action Mailed',
 'Response to Non-Final Office Action Entered and Forwarded to Examiner',
 'Patent Expired Due to NonPayment of Maintenance Fees Under 37 CFR 1.362',
 'Provisional Application Expired',
 'Expressly Abandoned  --  During Examination']

In [7]:
inprocess=[status[4],status[6],status[9],status[12],status[13],status[14],status[15],status[17]]
inprocess

['Docketed New Case - Ready for Examination',
 'Sent to Classification contractor',
 "Examiner's Answer to Appeal Brief Mailed",
 'Appeal Brief (or Supplemental Brief) Entered and Forwarded to Examiner',
 'Application Dispatched from Preexam, Not Yet Docketed',
 'RO PROCESSING COMPLETED-PLACED IN STORAGE',
 'PCT - International Search Report Mailed to IB',
 'Application Undergoing Preexam Processing']

In [8]:
objectdata['result']=[None for i in range(len(objectdata['status']))]

In [9]:
for i in range(len(objectdata['status'])):
    if objectdata['status'][i] in approved:
        objectdata['result'][i]=0
    elif objectdata['status'][i] in disapproval:
        objectdata['result'][i]=1
    else:
        objectdata['result'][i]=2

## Data Cleaning

In [10]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [11]:
def get_top_n_words(corpus, n=None,reverse=True):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=reverse)
    remwords= words_freq[:n]
    return [i[0] for i in remwords]

In [12]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    nonnumeric = re.sub(r"\d+","",normalized)
    processed = re.sub(r'\W*\b\w{1,4}\b',"",nonnumeric)
    #y = processed.split()
    return processed

In [13]:
objectdata['summary']=objectdata['summary'].apply(clean)
objectdata['title']=objectdata['title'].apply(clean)

In [14]:
common_words = get_top_n_words(objectdata['summary']+objectdata['title'],30)

In [15]:
common_words

['lidar',
 'system',
 'laser',
 'signal',
 'light',
 'method',
 'point',
 'target',
 'optical',
 'object',
 'source',
 'pulse',
 'image',
 'sensor',
 'using',
 'includes',
 'measurement',
 'first',
 'second',
 'detection',
 'device',
 'surface',
 'système',
 'position',
 'detector',
 'invention',
 'array',
 'information',
 'configured',
 'reflected']

In [16]:
def removewords(doc):
    word_free = " ".join([i for i in doc.split() if i not in common_words])
    return word_free

In [17]:
objectdata['summary']=objectdata['summary'].apply(removewords)
objectdata['title']=objectdata['title'].apply(removewords)

In [18]:
objectdata['filingDate']=objectdata['filingDate'].apply(lambda x: np.NaN if (len(x)<1) else x)
objectdata['filingDate']=objectdata['filingDate'].apply(lambda x: datetime.datetime.strptime(x, "%a %b %d %H:%M:%S %z %Y") if isinstance(x,str) else x) 
objectdata=objectdata.sort_values(by='filingDate')
objectdata=objectdata.reset_index(drop=True)

In [19]:
objectdata

,id,title,summary,status,filingDate,publicationNumber,publicationDate,objectType,result
0,06248629,removal background backscattering subtraction ...,,Patent Expired Due to NonPayment of Maintenanc...,1981-03-27 13:00:00+00:00,NaN,,patent,1
1,07146093,control photomultiplier detecting differential...,,Patented Case,1988-01-20 13:00:00+00:00,NaN,,patent,0
2,07216341,imaging,,Patented Case,1988-07-07 11:00:00+00:00,NaN,,patent,0
3,07256778,imaging nonvisible,,Patent Expired Due to NonPayment of Maintenanc...,1988-10-12 11:00:00+00:00,NaN,,patent,1
4,07348746,,,Expressly Abandoned -- During Examination,1989-05-08 11:00:00+00:00,NaN,,patent,1
5,07364860,incorporating multiple camera obtaining plural...,,Patent Expired Due to NonPayment of Maintenanc...,1989-06-12 11:00:00+00:00,NaN,,patent,1
6,07416652,arrangement measuring atmospheric turbidity,,Patented Case,1989-10-03 11:00:00+00:00,NaN,,patent,0
7,07420247,imaging,,Patented Case,1989-10-12 11:00:00+00:00,NaN,,patent,0
8,07499068,scanning,,Patented Case,1990-03-23 13:00:00+00:00,NaN,,patent,0
9,07539230,midinfrared hydrocarbon,,Patented Case,1990-06-18 11:00:00+00:00,NaN,,patent,0


In [20]:
objectdata['title_sum'] = objectdata.title + objectdata.summary
objectdata.drop(['id','title','summary','status','filingDate','publicationNumber','publicationDate','objectType'],axis=1, inplace=True)

In [21]:
vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = vectorizer.fit_transform(objectdata['title_sum'])
labels = objectdata['result']
features.shape

(358, 12062)

In [22]:
objectdata.head()

,result,title_sum
0,1,removal background backscattering subtraction ...
1,0,control photomultiplier detecting differential...
2,0,imaging
3,1,imaging nonvisible
4,1,


## Choosing a Classifier

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [24]:
models = [
    LogisticRegression(random_state=0, C=0.9),
    RandomForestClassifier(n_estimators=400, max_depth=50, random_state=0),
    XGBClassifier(),
    MultinomialNB(alpha=0.01)
]

In [25]:
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, list(labels), scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/Users/ankit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/ankit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [26]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LogisticRegression        0.485989
MultinomialNB             0.391213
RandomForestClassifier    0.593390
XGBClassifier             0.576605
Name: accuracy, dtype: float64

## Evaluation of RandomForestClassifier using Accuracy metric

In [27]:
trainX, testX, trainy, testy = train_test_split(features, list(labels), test_size=0.3, random_state=2)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score,mean_squared_error,roc_curve,auc
from matplotlib import pyplot as matplot_plt

# Function for evaluation reports
def get_eval(clf, X_train, y_train,y_test,y_pred):
    
    # Cross Validation to test and anticipate overfitting problem
    scores1 = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')

    # The mean score and standard deviation of the score estimate
    print("Resultant Cross Validation Accuracy will be: %0.2f (+/- %0.2f)" % (scores1.mean(), scores1.std()))
    
    # Create and print confusion matrix
    print("Confusion Matrix will be\n")
    confusion_res = confusion_matrix(y_test,y_pred)
    
    
    #Results Computed------

    print(confusion_res)
    print("\n")
    print("Classification report\n",classification_report(y_test, y_pred))
    return

In [29]:
def randomforest_evaluate(X_train, y_train,X_test, y_test):
    print("RandomForestClassifier")
    randomforest=RandomForestClassifier(n_estimators=400, max_depth=50, random_state=0).fit(X_train, y_train)

    # Predict target variables y for test data
    y_pred = randomforest.predict(X_test)
    
    # Get Cross Validation and Confusion matrix
    get_eval(randomforest, X_train, y_train,y_test,y_pred)
    return

In [30]:
randomforest_evaluate(trainX,trainy,testX,testy)

RandomForestClassifier
Resultant Cross Validation Accuracy will be: 0.59 (+/- 0.12)
Confusion Matrix will be

[[27 19  0]
 [11 36  0]
 [ 5  2  8]]


Classification report
               precision    recall  f1-score   support

           0       0.63      0.59      0.61        46
           1       0.63      0.77      0.69        47
           2       1.00      0.53      0.70        15

   micro avg       0.66      0.66      0.66       108
   macro avg       0.75      0.63      0.66       108
weighted avg       0.68      0.66      0.66       108



In [31]:
model=RandomForestClassifier(n_estimators=400, max_depth=50, random_state=0).fit(features,list(labels))

## Update Model

In [32]:
Updatecounter=0
def update_model():
    objectdata['title']=objectdata['title'].apply(clean)
    objectdata['summary']=objectdata['summary'].apply(clean)
    objectdata['title']=objectdata['title'].apply(removewords)
    objectdata['summary']=objectdata['summary'].apply(removewords)
    objectdata['title_sum'] = objectdata.title + objectdata.summary
    objectdata.drop(['id','title','summary','status','filingDate','publicationNumber','publicationDate','objectType'],axis=1, inplace=True)
    features=vectorizer.fit_transform(objectdata['title_sum'])
    labels=objectdata['result']
    model=RandomForestClassifier(n_estimators=400, max_depth=50, random_state=0).fit(features,list(labels))

## APIS

In [33]:
from flask import Flask,request
  
app = Flask(__name__) 
  
@app.route('/patentpredict', methods=["POST"]) 
def predict_patent(): 
    req = request.get_json()
    doc=pd.DataFrame(req['object'],index=[0])
    doc['title']=doc['title'].apply(clean)
    doc['summary']=doc['summary'].apply(clean)
    doc['title']=doc['title'].apply(removewords)
    doc['summary']=doc['summary'].apply(removewords)
    doc['title_sum'] = doc.title + doc.summary
    doc.drop(['id','title','summary','status','filingDate','publicationNumber','publicationDate','objectType'],axis=1, inplace=True)
    doc_titsum=vectorizer.transform(doc['title_sum'])
    res=model.predict(doc_titsum)
    if res[0]==0:
        return "Approved"
    elif res[0]==1:
        return "Not approved"
    else:
        return "In review"

@app.route('/patentrecord',methods=["POST"])
def record_patent():
    req = request.get_json()
    doc=pd.DataFrame(req['object'],index=[0])
    if doc['status'] in approved:
        doc['result']=[0]
    elif doc['status'] in disapproval:
        doc['result']=[1]
    else:
        doc['result']=2
    objectdata.append(doc)
    
    doc['title']=doc['title'].apply(clean)
    doc['summary']=doc['summary'].apply(clean)
    doc['title']=doc['title'].apply(removewords)
    doc['summary']=doc['summary'].apply(removewords)
    doc['title_sum'] = doc.title + doc.summary
    doc.drop(['id','title','summary','status','filingDate','publicationNumber','publicationDate','objectType'],axis=1, inplace=True)
    doc_titsum=vectorizer.transform(doc['title_sum'])
    res=model.predict(doc_titsum)
    if res[0]!=doc['result'][0]:
        Updatecounter+=1
    if Updatecounter>5:
        update_model()
    return "Submitted"
        
    
if __name__ == '__main__': 
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
